In [18]:
import pandas as pd
import os
import warnings
import pymongo
import json

pd.options.mode.chained_assignment = None
warnings.filterwarnings('ignore', category=RuntimeWarning)

for i in range(1): # one year only
    year = 2007
    print("YEAR: ", year)
    
    file_path = f"results/inputs_csv_zscore_last/PrixCarburants_annuel_filtered_{year}.csv"
    
    if not os.path.exists(file_path):
        print(f"Le fichier {file_path} n'existe pas.")
    else:
        df = pd.read_csv(file_path)
        df['date'] = pd.to_datetime(df['date'], format='%Y_%m_%d')
        df['heuremin'] = pd.to_datetime(df['heuremin'], format='%H:%M').dt.time
        print(df.head(5))
        print(df.dtypes)
        
        # Setup MongoDB connection (local)
        mongo_host = "mongodb://localhost"
        mongo_port = 27017
        client_mongo = pymongo.MongoClient(host=mongo_host, port=mongo_port)
        
        db_mongo = client_mongo.get_database("gas_station_price")
        collection_mongo = db_mongo.get_collection("gas_station_price")
        
        records = json.loads(df.to_json(orient="records"))
        collection_mongo.insert_many(records)
        print("correctly loaded", year, "csv to MongoDB")

YEAR:  2007
        id       date     nom   latitude  longitude    cp  \
0  1000001 2007-01-04  Gazole  4620114.0   519791.0  1000   
1  1000001 2007-01-04    SP95  4620114.0   519791.0  1000   
2  1000001 2007-01-09  Gazole  4620114.0   519791.0  1000   
3  1000001 2007-01-09    SP95  4620114.0   519791.0  1000   
4  1000001 2007-01-12  Gazole  4620114.0   519791.0  1000   

                   ville          adresse  valeur  heuremin  
0  SAINT-DENIS-LèS-BOURG  ROUTE NATIONALE   999.0  18:43:00  
1  SAINT-DENIS-LèS-BOURG  ROUTE NATIONALE  1170.0  18:43:00  
2  SAINT-DENIS-LèS-BOURG  ROUTE NATIONALE   999.0  08:32:00  
3  SAINT-DENIS-LèS-BOURG  ROUTE NATIONALE  1160.0  08:32:00  
4  SAINT-DENIS-LèS-BOURG  ROUTE NATIONALE   989.0  08:42:00  
id                    int64
date         datetime64[ns]
nom                  object
latitude            float64
longitude           float64
cp                    int64
ville                object
adresse              object
valeur              float